In [14]:
from bs4 import BeautifulSoup
from datetime import datetime, timezone, timedelta
import pandas as pd
import json
import requests
import numpy as np
import re
import time as time
import os
import json
import tensorflow as tf
from tensorflow import keras

In [99]:
np.random.seed(4)
tf.random.set_seed(4)

# Web Scraping
We are going to gather our data while putting minimal pressure on the target website's servers.

In [100]:
#Let's give our program some fake credentials to make it appear like a real person is using it
headers= {
    'accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,image/apng,*/*;q=0.8',
    'accept-encoding': 'gzip, deflate, br',
    'accept-language': 'en-US,en;q=0.9,fr;q=0.8,ro;q=0.7,ru;q=0.6,la;q=0.5,pt;q=0.4,de;q=0.3',
    'cache-control': 'max-age=0',
    'upgrade-insecure-requests': '1',
    'user-agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_13_6) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/67.0.3396.99 Safari/537.36'}

# These are the target url's which are mostly from rallys and speeches
url_list=[
    'https://www.rev.com/blog/transcripts/donald-trump-cpac-2021-speech-transcript',
    'https://www.rev.com/blog/transcripts/donald-trump-speech-transcript-departing-white-house-on-inauguration-day',
    'https://www.rev.com/blog/transcripts/president-donald-trump-farewell-address-speech-transcript',
    'https://www.rev.com/blog/transcripts/donald-trump-second-video-speech-condemning-capitol-violence-transcript',
    'https://www.rev.com/blog/transcripts/donald-trumps-first-comments-since-capitol-riots-says-he-wants-no-violence',
    'https://www.rev.com/blog/transcripts/donald-trump-concedes-election-condemns-rioters-video-transcript-january-7',
    'https://www.rev.com/blog/transcripts/trump-video-telling-protesters-at-capitol-building-to-go-home-transcript',
    'https://www.rev.com/blog/transcripts/donald-trump-speech-save-america-rally-transcript-january-6',
    'https://www.rev.com/blog/transcripts/donald-trump-rally-speech-transcript-dalton-georgia-senate-runoff-election',
    'https://www.rev.com/blog/transcripts/donald-trump-georgia-phone-call-transcript-brad-raffensperger-recording',
    'https://www.rev.com/blog/transcripts/donald-trump-hosts-operation-warp-speed-covid-19-vaccine-summit-transcript-december-8',
    'https://www.rev.com/blog/transcripts/donald-trump-georgia-rally-transcript-before-senate-runoff-elections-december-5',
    'https://www.rev.com/blog/transcripts/donald-trump-speech-on-election-fraud-claims-transcript-december-2',
    'https://www.rev.com/blog/transcripts/donald-trump-remarks-transcript-pennsylvania-republican-hearing-on-2020-election',
    'https://www.rev.com/blog/transcripts/donald-trump-white-house-press-conference-on-lowering-drug-prices-transcript',
    'https://www.rev.com/blog/transcripts/donald-trump-white-house-press-conference-as-election-counts-continue-transcript-november-5',
    'https://www.rev.com/blog/transcripts/donald-trump-rally-speech-transcript-kenosha-wi-november-2',
    'https://www.rev.com/blog/transcripts/donald-trump-rally-speech-transcript-scranton-pa-november-2',
    'https://www.rev.com/blog/transcripts/donald-trump-rally-speech-transcript-fayetteville-nc-november-2',
    'https://www.rev.com/blog/transcripts/donald-trump-rally-speech-transcript-rome-georgia-november-1',
    'https://www.rev.com/blog/transcripts/donald-trump-rally-speech-transcript-waterford-township-michigan-october-30',
    'https://www.rev.com/blog/transcripts/donald-trump-rally-speech-transcript-green-bay-wisconsin-october-30',
    'https://www.rev.com/blog/transcripts/donald-trump-rally-speech-transcript-tampa-fl-october-29',
    'https://www.rev.com/blog/transcripts/donald-trump-rally-speech-transcript-bullhead-city-az-october-28',
    'https://www.rev.com/blog/transcripts/donald-trump-rally-speech-transcript-omaha-ne-october-27',
    'https://www.rev.com/blog/transcripts/donald-trump-rally-speech-transcript-west-salem-wisconsin-october-27',
    'https://www.rev.com/blog/transcripts/donald-trump-rally-speech-transcript-lansing-michigan-october-27',
    'https://www.rev.com/blog/transcripts/donald-trump-circleville-ohio-rally-speech-transcript-october-24'
]

In [70]:
#Our web-scraping function which returns the statements made solely by Donald Trump.
def get_speeches(url_list):
    global string_all #A global variable containing all of strings, for greater flexibility in the future.
    string_all=''
    string_Trump=''
    for url in url_list:
        time.sleep(abs(np.random.normal(28,24))) #I don't want to add extra pressure on their servers. I also don't want anyone else running this code to get their IP address blocked, so I
                                                  #added randomness to the timing of our execution via a normal distribution

        html=requests.get('{}'.format(url), headers=headers).text #Get the html data
        soup=BeautifulSoup(html, 'html.parser')

        #Let's now get the main blocks of data which are the transcripts:
        transcript=soup.find_all('p')

        #If the block of text starts with some variation of Donald Trump:, then:
        transcript_DT=[statement for statement in transcript if ('Donald Trump:' in statement.text[:30]) | 
                ('President Trump:' in statement.text[:30]) | ('Donald J. Trump:' in statement.text[:35])] 
        
        for statement in transcript_DT:
            string_Trump+='\n '+re.split('(\n)', statement.text)[-1] #Use regex to split the sentences on \n which proceeds Trump:, retrieve Trump's speech. Example given below
        for statement in transcript:
            string_all+= statement.text

        print('{}% completed'.format( round( 100*((url_list.index(url)+1)/len(url_list)), 1) ))
    return string_Trump

In [71]:
string_Trump=get_speeches(url_list)

3.6% completed
7.1% completed
10.7% completed
14.3% completed
17.9% completed
21.4% completed
25.0% completed
28.6% completed
32.1% completed
35.7% completed
39.3% completed
42.9% completed
46.4% completed
50.0% completed
53.6% completed
57.1% completed
60.7% completed
64.3% completed
67.9% completed
71.4% completed
75.0% completed
78.6% completed
82.1% completed
85.7% completed
89.3% completed
92.9% completed
96.4% completed
100.0% completed


In [75]:
#We can also load in the text directly, which I saved from earlier
def load_Trump_str():
    global string_Trump
    string_Trump=''
    with open('text_Donald.txt','r') as file:
        list_Trump=file.readlines()
    for line in list_Trump:
        string_Trump+=line

def write_Trump_str():
    global string_Trump
    with open('text_Donald.txt', 'w') as file:
        file.write(string_Trump)

In [77]:
tokenizer = keras.preprocessing.text.Tokenizer(char_level=True)
tokenizer.fit_on_texts(string_Trump)

In [78]:
tokenizer.sequences_to_texts([[20, 6, 9, 8, 3]])

[', n s r t']

In [102]:
max_id = len(tokenizer.word_index) # number of distinct characters
dataset_size = tokenizer.document_count # total number of characters

In [103]:
tokenizer.

61

In [84]:
[encoded] = np.array(tokenizer.texts_to_sequences([string_Trump])) - 1
train_size = dataset_size * 95 // 100
dataset = tf.data.Dataset.from_tensor_slices(encoded[:train_size])

n_steps = 147 #He uses quite lengthy sentences to make his point, so I will set this higher to account for more long term memory
window_length = n_steps + 1 # target = input shifted 1 character ahead
dataset = dataset.repeat().window(window_length, shift=1, drop_remainder=True)

In [87]:
dataset = dataset.flat_map(lambda window: window.batch(window_length))
batch_size = 32
dataset = dataset.shuffle(7600).batch(batch_size)
dataset = dataset.map(lambda windows: (windows[:, :-1], windows[:, 1:]))

In [88]:
dataset = dataset.map(lambda X_batch, Y_batch: (tf.one_hot(X_batch, depth=max_id), Y_batch))

In [89]:
dataset = dataset.prefetch(1)

In [90]:
for X_batch, Y_batch in dataset.take(1):
    print(X_batch.shape, Y_batch.shape)

(32, 147, 61) (32, 147)


In [91]:
first_time=datetime.now()

model = keras.models.Sequential([
    keras.layers.GRU(128, return_sequences=True, input_shape=[None, max_id],
                     #dropout=0.2, recurrent_dropout=0.2),
                     dropout=0.2),
    keras.layers.GRU(128, return_sequences=True,
                     #dropout=0.2, recurrent_dropout=0.2),
                     dropout=0.2),
    keras.layers.TimeDistributed(keras.layers.Dense(max_id,
                                                    activation="softmax"))
])
model.compile(loss="sparse_categorical_crossentropy", optimizer="adam")
history = model.fit(dataset, steps_per_epoch=train_size // batch_size,
                    epochs=10)

time_end=datetime.now()-first_time
print(time_end)

Epoch 1/10
30962/30962 [==============================] - 13575s 438ms/step - loss: 1.5204
Epoch 2/10
30962/30962 [==============================] - 13923s 450ms/step - loss: 1.2908
Epoch 3/10
30962/30962 [==============================] - 13993s 452ms/step - loss: 1.2624
Epoch 4/10
30962/30962 [==============================] - 13996s 452ms/step - loss: 1.2479
Epoch 5/10
30962/30962 [==============================] - 14021s 453ms/step - loss: 1.2379
Epoch 6/10
30962/30962 [==============================] - 14032s 453ms/step - loss: 1.2323
Epoch 7/10
30962/30962 [==============================] - 14040s 453ms/step - loss: 1.2274
Epoch 8/10
30962/30962 [==============================] - 14103s 455ms/step - loss: 1.2239
Epoch 9/10
30962/30962 [==============================] - 14071s 454ms/step - loss: 1.2211
Epoch 10/10
30962/30962 [==============================] - 14040s 453ms/step - loss: 1.2182
1 day, 14:49:53.895391


In [93]:
model.save('TrumpBot.h5')

In [104]:
def preprocess(texts):
    X = np.array(tokenizer.texts_to_sequences(texts)) - 1
    return tf.one_hot(X, max_id)

In [105]:
X_new = preprocess(["How are yo"])
#Y_pred = model.predict_classes(X_new)
Y_pred = np.argmax(model(X_new), axis=-1)
tokenizer.sequences_to_texts(Y_pred + 1)[0][-1]

'u'

In [106]:
tf.random.set_seed(42)
tf.random.categorical([[np.log(0.5), np.log(0.4), np.log(0.1)]], num_samples=40).numpy()

array([[0, 1, 0, 2, 1, 0, 0, 1, 0, 0, 1, 1, 0, 1, 1, 1, 0, 2, 1, 0, 2, 1,
        0, 1, 2, 1, 1, 1, 2, 0, 0, 1, 0, 0, 1, 0, 1, 0, 0, 2]])

In [107]:
def next_char(text, temperature=1):
    X_new = preprocess([text])
    y_proba = model(X_new)[0, -1:, :]
    rescaled_logits = tf.math.log(y_proba) / temperature
    char_id = tf.random.categorical(rescaled_logits, num_samples=1) + 1
    return tokenizer.sequences_to_texts(char_id.numpy())[0]

In [108]:
tf.random.set_seed(42)

next_char("How are yo", temperature=1)

'u'

In [109]:
def complete_text(text, n_chars=50, temperature=1):
    for _ in range(n_chars):
        text += next_char(text, temperature)
    return text

In [116]:
speech_t=complete_text("Thank",n_chars=400, temperature=0.2)
print(speech_t)

In [117]:
print(speech)

Thank you very much. and i want to thank you than a journey will be a lot of things that we did a good job. i wouldn’t have run. we had a good job. i want to thank you were that nobody else is the state of our country. i said, “well, he was the state of our country. i said, “what the hell is the crimisal country. he wants to complete what it’s going to be that the most important election in the world. 


In [122]:
speech_t=complete_text("Biden",n_chars=133, temperature=0.2)
print(speech_t)

Biden has to come back. we have the criminals. the fake news and a half of the world. and it’s amazing the biden and a half. you know what
